# GAN

In [1]:
import os as os

In [2]:
import tensorflow as tf
import tensorflow.keras as keras

In [3]:
# Módulos necesarios. Asegurarse de poder importarlos.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import sys
from six.moves import cPickle as pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import csv
from pylab import rcParams
from scipy import stats
from sklearn import metrics
import seaborn as sns
from sklearn import preprocessing
import tqdm as tqdm

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [4]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
tf.__version__

/usr/bin/python
3.6.8 (default, Aug 20 2019, 17:12:48) 
[GCC 8.3.0]
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)


'1.14.0'

In [5]:
keras.__version__

'2.2.4-tf'

In [6]:
global activities
activities =  [19]
global subjects
subjects = [1,2,3,5,8,9,10,11,13,14,16,17]

In [7]:
# Ruta raíz
PATH = '../data_reset_def/'

quaturls = !ls -1 "{PATH}"



quat_corr = []
for filename in quaturls:
    for activity in activities:
        if(int(filename[-6:-4])==activity):
            quat_corr.append(filename)

quat_def = []

i=0
for filename in quat_corr:
    for subject in subjects:
        if(int(quat_corr[i][8:10])==subject):
            quat_def.append(filename)
    i+=1

quaturls = quat_def

del quat_corr, quat_def

n = len(quaturls)

tr_urls = quaturls

print('Total files: ' + str(len(quaturls)))
print('Train files: ' + str(len(tr_urls)))

tr_fullpath = [os.path.join(PATH,s) for s in tr_urls]
print('TRAINING: ')
print(str(tr_fullpath[:10]) + ' ...')

Total files: 12
Train files: 12
TRAINING: 
['../data_reset_def/subject_01_RESET_act_19.csv', '../data_reset_def/subject_02_RESET_act_19.csv', '../data_reset_def/subject_03_RESET_act_19.csv', '../data_reset_def/subject_05_RESET_act_19.csv', '../data_reset_def/subject_08_RESET_act_19.csv', '../data_reset_def/subject_09_RESET_act_19.csv', '../data_reset_def/subject_10_RESET_act_19.csv', '../data_reset_def/subject_11_RESET_act_19.csv', '../data_reset_def/subject_13_RESET_act_19.csv', '../data_reset_def/subject_14_RESET_act_19.csv'] ...


### Balanceo entre clases

Para el balanceo de clases forzamos a que cada clase tenga el mismo número de muestras, así que, en este caso, limitamos el nº de muestras al que tiene el mínimo (en el futuro se hará data augmentation):

In [8]:
n = 1e100
for filename in tr_fullpath:
    df = pd.read_csv(filename,sep=',') 
    
    ind = len(df.index)
    if(ind!=0):
        n = min(n,ind)

ind = int(n)
if(PATH == '../data_augment/'):
    ind_real = 24*int(n/9)
else:
    ind_real = int(n/9)
print(n)

8928


In [9]:
print('Utilizamos: '+ str(ind_real) + ' muestras de cada actividad de cada sujeto, que se corresponde con: '+ str(ind_real/50) + ' segundos.')

Utilizamos: 992 muestras de cada actividad de cada sujeto, que se corresponde con: 19.84 segundos.


### Generación del dataset

In [10]:
def sliding_window(df, n_time_steps, step, segments, labels, label, dim, train, n_channels):
    quat0 = df.iloc[:, 1:5][df['QUAT']=='quat0'].reset_index() # si no incluimos el reset_index(), al concatenarlos después
    quat1 = df.iloc[:, 1:5][df['QUAT']=='quat1'].reset_index() # aparecerá un dataframe de igual nº de filas que df, pero con
    quat2 = df.iloc[:, 1:5][df['QUAT']=='quat2'].reset_index() # NaN en las posiciones que no tienen número de cada dataframe quat,
    quat3 = df.iloc[:, 1:5][df['QUAT']=='quat3'].reset_index() # es decir, mantiene los índices de df.
    quat4 = df.iloc[:, 1:5][df['QUAT']=='quat4'].reset_index()
    
    quat = pd.concat([quat0.iloc[:, 1], quat0.iloc[:, 2], quat0.iloc[:, 3], quat0.iloc[:, 4],
                      quat1.iloc[:, 1], quat1.iloc[:, 2], quat1.iloc[:, 3], quat1.iloc[:, 4],
                      quat2.iloc[:, 1], quat2.iloc[:, 2], quat2.iloc[:, 3], quat2.iloc[:, 4],
                      quat3.iloc[:, 1], quat3.iloc[:, 2], quat3.iloc[:, 3], quat3.iloc[:, 4],
                      quat4.iloc[:, 1], quat4.iloc[:, 2], quat4.iloc[:, 3], quat4.iloc[:, 4]],
                      axis = 1, keys = ['w0', 'x0', 'y0', 'z0', 'w1', 'x1', 'y1', 'z1', 'w2', 'x2', 'y2', 'z2', 
                                        'w3', 'x3', 'y3', 'z3', 'w4', 'x4', 'y4', 'z4'])
    
    del quat0, quat1, quat2, quat3, quat4
    
    if(n_channels == 1):
        for i in range(0, quat.shape[0] - n_time_steps, step): # Overlap
            # Con listas y numpy
            segments.append([])
            segments[dim].append(quat.iloc[i: i + n_time_steps, :].values) # Si distinguimos entre sensores
            labels.append(label-1)
            dim+=1
    else:
        n_columns = int(36/n_channels)
        for i in range(0, quat.shape[0] - n_time_steps, step):
            segments.append([])
            col = 0
            for j in range(n_channels):
                segments[dim].append([])
                segments[dim][j].append(quat.iloc[i:i+n_time_steps,col:col+n_columns].values)
                col+=n_columns
            labels.append(label-1)
            dim+=1
    
        
    del quat
    
    return segments, labels, dim


def load_quat(path, lim, train, n_channels):

    n_time_steps, step = 128, 1  # n_time_steps/50 segundos de actividad y pasos de step/50 segundos (de overlap)
    
    # Con listas
    segments = []
    labels = []
    
    m=1
    dim = 0
    for filename in path:
        print("Reading %s (%d/%d)                                                   "%(filename, m, len(path)), end='\r')
        df = pd.read_csv(filename,sep=',',names=["QUAT","w","x","y","z","timestamp"])
        label = int(filename[-6:-4])
        for i in range(len(activities)):
            if(label==activities[i]):
                label=i+1
        
        segments, labels, dim = sliding_window(df.iloc[:lim], n_time_steps, step, segments, labels, label, dim, train, n_channels)
        
        m+=1
    
    del df
    
    return segments, labels


def load_train_quat(filename, lim, n_channels):
    return load_quat(filename, lim, True, n_channels)

def load_test_quat(filename, lim, n_channels):
    return load_quat(filename, lim, False, n_channels)

def get_dataset(data_path, lim, batch_size, n_time_steps, train, valid, ds, n_channels):
    if(train):
        segments, labels = load_train_quat(data_path, lim, n_channels)
    else:
        segments, labels = load_test_quat(data_path, lim, n_channels)
    
    print('Generating the dataset                                                   ')                                              
    
    array = np.asarray(segments, dtype = 'float32')
    segments = np.reshape(array, (array.shape[0], n_channels, n_time_steps, int(20/n_channels)))
    array = np.asarray(labels, dtype = 'int8')
    labels = np.reshape(array, (array.shape[0], 1))
    
    del array
    '''
    # Standardization
    if(train):
        mean = segments.mean(axis=0)
        std = segments.std(axis=0)
        segments -= mean
        segments /= std
    else:
        segments -= mean
        segments /= std
    '''
    # Map coninous dataset to categorical (One-Hot)
    #labels = keras.utils.to_categorical(labels, 5)
    
    if(train):
        print('-'*20 + 'TRAIN' + '-'*20)
    elif(valid):
        print('-'*18 + 'VALIDATION' + '-'*12)
    else:
        print('-'*20 + 'TEST' + '-'*21)
    
    if(ds):
        dataset = tf.data.Dataset.from_tensor_slices((segments, labels))
    
        # It's necessary to repeat our data for all epochs
        dataset = dataset.batch(batch_size)
        
        dataset = dataset.shuffle(segments.shape[0])
        
        print('Dataset generated                                                        ')
        
        return dataset, segments, labels
    else:
        # Shuffle in the first dimension
        permutation = np.arange(0,segments.shape[0]-1)
        np.random.shuffle(permutation)
        segments = segments[permutation]
        labels = labels[permutation]
        print('Dataset generated                                                        ')
        
        return segments, labels

In [11]:
n_time_steps = 128
dataset = False # Set to True if you want a dataset or to False if you want np.arrays
batch_size = 16 # REAL batch_size
n_channels = 1 # It can be 1,4 or 9

if(dataset):
    train_dataset, tr_seg, tr_lab = get_dataset(tr_fullpath, ind, batch_size, n_time_steps, True, False, dataset, n_channels)
    
    print('Train dataset: ')
    print(train_dataset)
else:
    tr_seg, tr_lab = get_dataset(tr_fullpath, ind, batch_size, n_time_steps, True, False, dataset, n_channels)
    print('Train dataset: ')
    print(tr_seg.shape, tr_lab.shape)

Generating the dataset                                                                                          
--------------------TRAIN--------------------
Dataset generated                                                        
Train dataset: 
(10367, 1, 128, 20) (10367, 1)


In [12]:
def load_data():
    
    return tr_seg, tr_lab

# GAN

In [13]:
def adam_optimizer():
    return keras.optimizers.Adam(lr=0.0002, beta_1=0.5)

In [21]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

def create_generator():
    generator=Sequential()

    units = 1000
    
    generator.add(Reshape((-1,units), input_shape=(1000,)))
    
    generator.add(GRU(units, activation='relu', reset_after = False))
    #generator.add(Reshape((-1,units)))
    
    #generator.add(GRU(units, activation='relu', dropout=0.5, recurrent_dropout=0.5, reset_after = False))
    
    generator.add(Dense(units=tr_seg.shape[1]*tr_seg.shape[2]*tr_seg.shape[3], activation='linear'))
    
    generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return generator
g=create_generator()
g.summary()
#keras.utils.plot_model(g, to_file="model.png", show_shapes=True, show_layer_names=True)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_12 (Reshape)         (None, None, 1000)        0         
_________________________________________________________________
gru_8 (GRU)                  (None, 1000)              6003000   
_________________________________________________________________
dense_12 (Dense)             (None, 2560)              2562560   
Total params: 8,565,560
Trainable params: 8,565,560
Non-trainable params: 0
_________________________________________________________________


In [22]:
def create_discriminator():
    discriminator=Sequential()

    discriminator.add(Reshape((tr_seg.shape[1],tr_seg.shape[2],tr_seg.shape[3]), input_shape=(2560,)))
    
    discriminator.add(Conv2D(64,kernel_size=4, strides=1, padding='same', data_format='channels_first', activation='relu'))
    discriminator.add(MaxPooling2D(pool_size=4, strides=4, data_format='channels_first'))
    discriminator.add(Dropout(0.5))
       
    discriminator.add(Conv2D(128,kernel_size=4, strides=1, padding='same', data_format='channels_first', activation='relu'))
    discriminator.add(MaxPooling2D(pool_size=4, strides=4, data_format='channels_first'))
    discriminator.add(Dropout(0.5))
    
    discriminator.add(Flatten())
    
    discriminator.add(Dense(units=512, activation='relu'))
    
    discriminator.add(Dense(units=1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return discriminator
d =create_discriminator()
d.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_13 (Reshape)         (None, 1, 128, 20)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 128, 20)       1088      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 64, 32, 5)         0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 64, 32, 5)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 128, 32, 5)        131200    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 128, 8, 1)         0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 128, 8, 1)        

In [23]:
def create_gan(discriminator, generator):
    discriminator.trainable=False
    gan_input = Input(shape=(1000,))
    x = generator(gan_input)
    gan_output= discriminator(x)
    gan= Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan
gan = create_gan(d,g)
gan.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1000)]            0         
_________________________________________________________________
sequential_8 (Sequential)    (None, 2560)              8565560   
_________________________________________________________________
sequential_9 (Sequential)    (None, 1)                 657601    
Total params: 9,223,161
Trainable params: 8,565,560
Non-trainable params: 657,601
_________________________________________________________________


In [24]:
def write(name, ts_seg):
    fo = open(name, "w")
    head = "QUAT,w,x,y,z,timestamp\n"
    fo.seek(0,2)
    fo.write(head)
    fo.close()
    
    fo = open(name, "a")
    salida = np.reshape(ts_seg, (ts_seg.shape[0]*ts_seg.shape[2], ts_seg.shape[3]))
    
    k = 0
    for i in range(salida.shape[0]):
        if(i%128==0):
            if(i!=0):
                k+=1
        print('%d'%(i), end='\r')
        fo.write("quat0,"+str(salida[i][0])+","+str(salida[i][1])+","+str(salida[i][2])+","+str(salida[i][3])+","+str(i)+","+'\n')
        fo.write("quat1,"+str(salida[i][4])+","+str(salida[i][5])+","+str(salida[i][6])+","+str(salida[i][7])+","+str(i)+","+'\n')
        fo.write("quat2,"+str(salida[i][8])+","+str(salida[i][9])+","+str(salida[i][10])+","+str(salida[i][11])+","+str(i)+","+'\n')
        fo.write("quat3,"+str(salida[i][12])+","+str(salida[i][13])+","+str(salida[i][14])+","+str(salida[i][15])+","+str(i)+","+'\n')
        fo.write("quat4,"+str(salida[i][16])+","+str(salida[i][17])+","+str(salida[i][18])+","+str(salida[i][19])+","+str(i)+","+'\n')
        if(salida.shape[1]==36):
            fo.write("quat5,"+str(salida[i][20])+","+str(salida[i][21])+","+str(salida[i][22])+","+str(salida[i][23])+","+str(i)+","+'\n')
            fo.write("quat6,"+str(salida[i][24])+","+str(salida[i][25])+","+str(salida[i][26])+","+str(salida[i][27])+","+str(i)+","+'\n')
            fo.write("quat7,"+str(salida[i][28])+","+str(salida[i][29])+","+str(salida[i][30])+","+str(salida[i][31])+","+str(i)+","+'\n')
            fo.write("quat8,"+str(salida[i][32])+","+str(salida[i][33])+","+str(salida[i][34])+","+str(salida[i][35])+","+str(i)+","+'\n')
        else:
            fo.write("quat5,"+str(1)+","+str(0)+","+str(0)+","+str(0)+","+str(i)+","+'\n')
            fo.write("quat6,"+str(1)+","+str(0)+","+str(0)+","+str(0)+","+str(i)+","+'\n')
            fo.write("quat7,"+str(1)+","+str(0)+","+str(0)+","+str(0)+","+str(i)+","+'\n')
            fo.write("quat8,"+str(1)+","+str(0)+","+str(0)+","+str(0)+","+str(i)+","+'\n')
    fo.close()

In [ ]:
from IPython.display import clear_output

def training(epochs=1, batch_size=128):
    
    #Loading the data
    tr_seg, _ = load_data()
    batch_count = tr_seg.shape[0] // batch_size
    X_train = np.reshape(tr_seg,(tr_seg.shape[0],tr_seg.shape[2]*tr_seg.shape[3]))
    
    # Creating GAN
    generator= create_generator()
    discriminator= create_discriminator()
    gan = create_gan(discriminator, generator)
    
    d_real_hist = []
    d_fake_hist = []
    a_hist = []

    for e in range(1,epochs+1 ):
        print("\n Epoch %d" %e)
        for i in range (batch_count):
        #generate  random noise as an input  to  initialize the  generator
            noise= np.random.normal(0,1, [batch_size, 1000])
            
            #Pre train discriminator on  fake and real data  before starting the gan. 
            discriminator.trainable=True
            
            # Get a random set of  real images
            image_batch =X_train[np.random.randint(low=0,high=X_train.shape[0],size=batch_size)]
            y_dis=np.ones(batch_size)
            
            d_loss_real=discriminator.train_on_batch(image_batch, y_dis)
            
            # Generate fake MNIST images from noised input
            generated_images = generator.predict(noise)
            y_dis=np.zeros(batch_size)
            
            d_loss_fake=discriminator.train_on_batch(generated_images, y_dis)
            
            #Tricking the noised input of the Generator as real data
            noise= np.random.normal(0,1, [batch_size, 1000])
            y_gen = np.ones(batch_size)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #training  the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            # keras.utils.to_categorical(y_gen, 2)
            a_loss=gan.train_on_batch(noise,  y_gen)
            
            d_real_hist.append(d_loss_real)
            d_fake_hist.append(d_loss_fake)
            a_hist.append(a_loss)
            
            log_mesg = "%d: [D loss REAL : %f]" % (i+1, d_loss_real)
            log_mesg = "%s  [D loss FAKE: %f]" % (log_mesg, d_loss_fake)
            log_mesg = "%s  [A loss: %f]" % (log_mesg, a_loss)
            print(log_mesg, end='\r')
            
        if e == 1 or e % 100 == 0:
            name = "./00_10_models/generated_00_10_%s.csv"%(e)
            write(name, np.reshape(generated_images,(-1, tr_seg.shape[1], tr_seg.shape[2], tr_seg.shape[3])))
        
        clear_output(wait=True)
        
    return generator, discriminator, gan, d_real_hist, d_fake_hist, a_hist

epochs = 500
batch_size = 256
generator, discriminator, gan, d_real_hist, d_fake_hist, a_hist = training(epochs,batch_size)


 Epoch 288


In [ ]:
# Save model
generator.save('./00_10_models/generator.h5')
print('Generator saved.')
discriminator.save('./00_10_models/discriminator.h5')
print('Discriminator saved.')
gan.save('./00_10_models/GAN.h5')
print('GAN saved.')

In [ ]:
batch_count = tr_seg.shape[0] // batch_size

x_ticks = np.linspace(1,epochs,num=len(d_real_hist))

plt.figure(figsize=(30,10))
plt.plot(x_ticks,d_real_hist, label='Disc. loss REAL')
plt.plot(x_ticks,d_fake_hist, label='Disc. loss FAKE')
plt.plot(x_ticks,a_hist, label='Adv. loss')
plt.xlabel('Epochs', fontsize=20)
plt.ylabel('Losses', fontsize=20)
plt.legend(loc=1, prop={'size': 20})
plt.tick_params(labelsize=20);
plt.title('GAN Training', fontsize=20)

plt.draw()
plt.savefig('./00_10_models/training.tiff', bbox_inches='tight',format='tiff')
plt.draw()
plt.savefig('./00_10_models/training.pdf', bbox_inches='tight',format='pdf')

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(x_ticks,d_real_hist, label='Disc. loss REAL')
plt.plot(x_ticks,d_fake_hist, label='Disc. loss FAKE')
plt.xlabel('Epochs', fontsize=20)
plt.ylabel('Losses', fontsize=20)
plt.legend(loc=1, prop={'size': 20})
plt.tick_params(labelsize=20);
plt.title('GAN Training', fontsize=20)

plt.draw()
plt.savefig('./00_10_models/training_noAdv.tiff', bbox_inches='tight',format='tiff')
plt.draw()
plt.savefig('./00_10_models/training_noAdv.pdf', bbox_inches='tight',format='pdf')

In [ ]:
d_hist = np.asarray(d_real_hist)+np.asarray(d_fake_hist)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(x_ticks,d_hist, label='Disc. loss')
plt.plot(x_ticks,a_hist, label='Adv. loss')
plt.xlabel('Epochs', fontsize=20)
plt.ylabel('Losses', fontsize=20)
plt.legend(loc=1, prop={'size': 20})
plt.tick_params(labelsize=20)
plt.title('GAN Training', fontsize=20)

plt.draw()
plt.savefig('./00_10_models/training2.tiff', bbox_inches='tight',format='tiff')
plt.draw()
plt.savefig('./00_10_models/training2.pdf', bbox_inches='tight',format='pdf')

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(x_ticks,d_hist, label='Disc. loss')
plt.xlabel('Epochs', fontsize=20)
plt.ylabel('Losses', fontsize=20)
plt.legend(loc=1, prop={'size': 20})
plt.tick_params(labelsize=20)
plt.title('GAN Training', fontsize=20)

plt.draw()
plt.savefig('./00_10_models/training2_noAdv.tiff', bbox_inches='tight',format='tiff')
plt.draw()
plt.savefig('./00_10_models/training2_noAdv.pdf', bbox_inches='tight',format='pdf')

In [ ]:
print(d_hist[-1])

In [ ]:
print(d_real_hist[-1])
print(d_fake_hist[-1])